In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [2]:
%%sql
create or replace table competencia_03 as 
select *
  from read_csv_auto('/home/ms_beckel/buckets/b1/datasets/competencia_03_crudo.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [6]:
%sql select * from competencia_03 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,0.0,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.0,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,0.0,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,0.0,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66


In [7]:
%%sql
select foto_mes, 
       count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
  from competencia_03
 group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [3]:
%%sql
CREATE OR REPLACE TABLE COMPETENCIA2 AS

SELECT
        numero_de_cliente,
        foto_mes,
        --LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes2,
        --LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes3,
        CASE
            WHEN LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NOT NULL
                 AND LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NOT NULL
            THEN 'CONTINUA'

            WHEN LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NULL
            THEN 'BAJA+1'

            WHEN LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NOT NULL
                 AND LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NULL
            THEN 'BAJA+2'

        END AS clase_ternaria
 from competencia_03
 order by numero_de_cliente, foto_mes

,Success


In [4]:
%sql select * from COMPETENCIA2 limit 5

,numero_de_cliente,foto_mes,clase_ternaria
0,29183981,201901,CONTINUA
1,29183981,201902,CONTINUA
2,29183981,201903,CONTINUA
3,29183981,201904,CONTINUA
4,29183981,201905,CONTINUA


In [5]:
%%sql
CREATE OR REPLACE TABLE COMPETENCIA3 AS
select c1.*, c2.clase_ternaria from competencia_03 as c1 left join COMPETENCIA2 as c2 on c1.numero_de_cliente = c2.numero_de_cliente and c1.foto_mes = c2.foto_mes

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [6]:
%%sql
select * from COMPETENCIA3
limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
1,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00,CONTINUA
2,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31,CONTINUA
3,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66,CONTINUA
4,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,1,0.00,-5791.17,0.00,7827,12539.39,6,0,797.64,CONTINUA


In [7]:
%%sql
COPY COMPETENCIA3 TO '/home/ms_beckel/buckets/b1/datasets/competencia_03.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
